# Applied Digital Citizen Science
## Session 6

DISCLOSURE: Parts of the code for this notebook were created with GitHub CoPilot, and disclosed as such. All code has been tested by the lecturers.

See Canvas for details about usage of Generative AI in assignments

## Objectives for Session 6

1. Access the data storage
2. Read and consolidate the donated files into a (raw) dataset
3. Inspect, minimise and (if needed) anonymize the (raw) dataset
4. Save the processed dataset to your own computer

## Importing required libraries

In [ ]:
import os
import pandas as pd
import json

## Accessing the data storage

Please see on Canvas the details on data storage and data management rules. When ready, update the information on the variable "path", below.


In [ ]:
path = ""

## Consolidating the files

The code below uses the library "os" to list all files in the path you indicated above, and store them in a list called files.

In [ ]:
files = os.listdir(path)

In [ ]:
files

The following steps will create two datasets:
* Onboarding: Information about participant activity in the first page within Port (i.e., the general informed consent/about the study)
* YouTube: Data donation (and/or declining of consent) after data visualisation by participants.

Please note that you need to specify which "status" for the dataset you want, e.g., "test", "survey" or any other status code. The function will collect this information from the filename.

In [ ]:
def create_datasets(path, status):
    onboarding = pd.DataFrame()
    datasets = {}
    
    for file in files:
        status = file.split('_participant=')[1].split('_')[1]
        group = file.split('_participant=')[1].split('_')[0]
        participantid = file.split('_participant=')[1].split('_')[2]
        filetype = file.split('_source=')[1]

        if "onboarding" in filetype:
            tmp = pd.read_json(os.path.join(path, file), lines=True)
            tmp['group'] = group
            tmp['participantid'] = participantid
            onboarding = pd.concat([onboarding, tmp], ignore_index=True)

        if "YouTube" in filetype:
            yt = json.load(open(os.path.join(path, file), encoding='utf-8'))
            
            for item in yt:
                keys = item.keys()
                for key in keys:
                    tmp_yt = pd.json_normalize(item[key])
                    tmp_yt['group'] = group
                    tmp_yt['participantid'] = participantid
                    if key not in datasets:
                        datasets[key] = tmp_yt
                    else:
                        datasets[key] = pd.concat([datasets[key],tmp_yt], ignore_index=True)

    
    return onboarding, datasets

In [ ]:
onboarding, youtube = create_datasets(path, "test")

In [ ]:
onboarding.head()

In [ ]:
youtube.keys()

In [ ]:
youtube['youtube_kijkgeschiedenis']

In [ ]:
watch = youtube['youtube_kijkgeschiedenis']
search = youtube['youtube_zoekgeschiedenis']

## Inspecting the data

In [ ]:
onboarding.columns

In [ ]:
onboarding.head()

In [ ]:
onboarding.iloc[0]

In [ ]:
watch.columns

In [ ]:
watch.head()

In [ ]:
watch.iloc[0]

## Discussion: Anonymisation and Minimisation

## Exporting the datasets to our own computers

In [ ]:
watch.to_excel('watch.xlsx')
search.to_excel('search.xlsx')
onboarding.to_excel('logs.xlsx')